In [7]:
import os
import glob
import time
import numpy as np
from skimage import io
from skimage.transform import resize
import tensorflow as tf
from tqdm.auto import tqdm
from time import sleep
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [8]:
# path = "D:/adversarial_data/*.bmp"
path = 'data/adversarial_data/ionization_bubble/*.bmp'
save_dir = 'gan/outputs/basic_gan'
print("data path: ",path)
print("output directory: ",save_dir)

data path:  data/adversarial_data/ionization_bubble/*.bmp
output directory:  gan/outputs/basic_gan


In [9]:
filelist = glob.glob(path)

In [10]:
len(filelist)

0

In [11]:
img_height = 128
img_width = 128
channels = 1
batch = 14
epochs = 10000000
dims = 100

In [12]:
def preprocess(x, height, width):
    img = resize(x,(height, width))
    img /= 127.5-1
    return img

In [13]:
def gen_graph(dims=100, image_shape=(28,28,1)):
    
    gen_input = tf.keras.Input(shape=(dims,))
    x = tf.keras.layers.Dense(256)(gen_input)
    x = tf.keras.layers.LeakyReLU(0.3)(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.8)(x)
    x = tf.keras.layers.Dense(512)(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.8)(x)
    x = tf.keras.layers.Dense(np.prod(image_shape),activation='tanh')(x)
    x = tf.keras.layers.Reshape(image_shape)(x)
    
    generator = tf.keras.models.Model(gen_input, x)
    
    opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    generator.compile(loss='binary_crossentropy', optimizer=opt)
    generator.summary()
    
    return generator

In [14]:
def disc_graph(height=128,width=128,channel=1):

    disc_input = tf.keras.layers.Input(shape=(height,width,channel))
    x = tf.keras.layers.Flatten()(disc_input)
    x = tf.keras.layers.Dense(512)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    discriminator = tf.keras.models.Model(disc_input,x)

    opt = tf.keras.optimizers.Adam(lr=0.0002,beta_1=0.5)
    discriminator.compile(optimizer=opt,loss='binary_crossentropy')
    discriminator.summary()
    
    return discriminator

In [15]:
def adversarial_graph(disc_net, gen_net, dims=100):
    disc_net.trainable=False
    adv_input = tf.keras.Input(shape=(dims,))
    adv_output = disc_net(gen_net(adv_input))
    
    adv_network = tf.keras.Model(inputs=adv_input, outputs=adv_output)
    
    opt = tf.keras.optimizers.Adam(0.001,0.9)
    adv_network.compile(loss='binary_crossentropy', optimizer=opt)
    adv_network.summary()
    
    return adv_network

In [16]:
gen_model = gen_graph(image_shape=(img_height,img_width,channels))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batc

In [17]:
disc_model = disc_graph()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               8389120   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total para

In [18]:
adv_net = adversarial_graph(disc_model,gen_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model (Model)                (None, 128, 128, 1)       8565504   
_________________________________________________________________
model_1 (Model)              (None, 1)                 8520705   
Total params: 17,086,209
Trainable params: 8,563,968
Non-trainable params: 8,522,241
_________________________________________________________________


In [19]:
def load_data(filelist,height,width):
    X_train = []
    for i in filelist:
#         print(i)
        img = io.imread(i,as_gray=True)
        img= preprocess(img,height,width)
        X_train.append(img)
    X_train = np.array(X_train)
    X_train = np.reshape(X_train,(-1,height, width, 1))
    return X_train

In [20]:
gen = load_data(filelist,img_height,img_width)

In [27]:
filelist

[]

In [25]:
gen.shape[0]

0

In [21]:
def sample_images(epoch,dims,path):
        r, c = 5,5
        noise = np.random.normal(0,1,(r*c, dims))
        
        gen_imgs = gen_model.predict(noise)
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r,c)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[count,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                count += 1
        fig.savefig(path+'/%d.png'%epoch)
        plt.close()

In [22]:
def train(batch_size, epochs, k,X_train,latent_dims):
        
        
#     (X_train, _), (_,_) = mnist.load_data()
    # Rescale -1 to 1
#     X_train = X_train/ 127.5 - 1
#     X_train = np.expand_dims(X_train, axis=3)

    #ground truths
    real = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))

    for i in range(epochs):
        # Algorithm 1
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0,1,(batch_size, latent_dims))

        gen_imgs = gen_model.predict(noise)

        #Train the discriminator
        d_loss_real = disc_model.train_on_batch(imgs,real)
        d_loss_fake = disc_model.train_on_batch(gen_imgs, fake)
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake)
#         print(d_loss)
        #Train Generator
        noise = np.random.normal(0,1, (batch_size, latent_dims))
        g_loss = adv_net.train_on_batch(noise, real)
#         print(g_loss)
        #progress per epoch
        print("%d [D loss: %f, accuracy: %.2f%%] [G loss:%f]" % (i, d_loss, 100*d_loss, g_loss))

        if i % k == 0:
            sample_images(i,100,save_dir)

In [23]:
train(batch,epochs,100000,gen,100)

ValueError: low >= high

In [24]:
gen.shape[0]

0